In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


데이터 준비. 입력 변수(X)와 목표 변수(Y)

In [21]:
xData = [1, 2, 3, 4, 5, 6, 7] # 1일 근무시간
yData = [25000, 55000, 75000, 110000, 128000, 155000, 180000] # 매출금액, Y, label

과적합을 방지하기 위해서 난수를 발생시켜 기울기(가중치)와 y절편(바이어스)을 정한다.

과적합이란 학습시킨 데이터에서는 높은 정확도를 보이지만 이외의 데이터에서는 낮은 정확도를 보이는 문제점을 말한다.

In [3]:
# tf.random_uniform([난수의 개수], 최소값, 최대값): 난수를 발생시킨다.
a = tf.Variable(tf.random_uniform([1], -100, 100)) # 난수로 기울기(가중치) 정한다
b = tf.Variable(tf.random_uniform([1], -100, 100)) # 난수로 y절편(bias) 정한다
print('a={}, b={}'.format(a,b))

a=<tf.Variable 'Variable_2:0' shape=(1,) dtype=float32_ref>, b=<tf.Variable 'Variable_3:0' shape=(1,) dtype=float32_ref>


tensorflow 변수를 초기화하고 난수로 발생시킨 기울기와 y절편 값을 확인한다.

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer()) #  변수 초기화
print('a={}, b={}'.format(sess.run(a), sess.run(b)))

a=[-21.186234], b=[33.60498]


근무시간과 매출금액을 기억할 placeholder 선언.

placeholder

- 학습용 데이터를 담는 그릇으로, 실행 시점에 데이터를 할당
- tf.placeholder(dtype[, shape, name])

In [25]:
x = tf.placeholder(dtype=tf.float32) # 근무시간을 기억할 placeholder
y = tf.placeholder(dtype=tf.float32) # 매출금액을 기억할 placeholder


오차 vs 잔차
- 오차는 모집단의 회귀식에 대한 편차값인데 반해 잔차는 표본집단의 회귀식에 대한 편차값이다. 
- 즉, 오차는 관측값을 통해 예측한 가정이 실제와 얼마나 부합하는지
- 잔차는 예측한 가정이 관측값을 얼마나 잘 반영하고 있는가를 의미한다고 할 수 있다

모델링 
- 데이터 내의 패턴을 파악해 모형화하는 것
- 데이터를 잘 설명하는 수식을 생성하는 것

비용함수
- 모델의 예측값과 실제값의 차이 => 값이 클수록 정확도 낮음
- 실제값과 예측값(Y)의 편차 제곱의 평균.. 2차함수(x=기울기, y=에러)
- 에러가 최소인 기울기를 구한다 by 경사하강법

In [26]:
가설을 세우고 오차(비용) 함수를 정의한다.
Y=a*x+b # 가설, Y => 예측값(직선)

# 제곱평균법을 이용하는 오차 함수
# 실제값(yData => y라는 placeholder에 저장)과 예측값(Y)의 편차 제곱의 평균을 이용한다. # square(): 제곱값 # reduce_mean(): 평균

# 오차함수 정의
cost= tf.reduce_mean(tf.square(Y-y)) # 실제값과 예측값(Y)의 편차 제곱의 평균.

경사하강법 오차 함수의 결과를 가장 작게 만드는 방향으로 학습하도록 정의한다.

- 경사 하강법이란?
- 비용 함수가 최소가 되는 w, b를 구하는 방법으로, 점진적으로 반복적인 계산을 통해 w, b의 값을 업데이트해 오류 값이 최소가 되는 w, b를 구한다.
- 현재 위치에서 기울기를 따라 목표값을 향하여 계속 낮은 곳으로 반복 이동

경사 하강법은 다음의 방법으로 진행됩니다.

1. 임의의 한 점을 찍고 미분값(기울기)를 구한다. cost(w, b)를 계산한다. 
2. 접선의 기울기를 바탕으로 파라미터를 '어느 방향'으로 '얼만큼' 움직일지 결정한다. w, b를 cost(w, b)가 줄어드는 방향으로 값을 변경
3. 기울기=0인 지점까지 반복해서 이동한다. 기울기가 음수면 오른쪽, 양수면 왼쪽으로 이동

학습률(learning rate)
- 값을 식에 얼마큼 반영할 것인지 알려주는 변수입니다. 보통 0.01로 설정. 
- 너무 적게 설정하면 학습이 너무 느리게 진행되고 
- 너무 크게 설정하면 손실 함수의 값이 줄어들지 않고 더 커지게 되는 오버 슈팅(over shooting) 발생.

Optimizer
- 비용 함수를 최소화하기 위해 조금씩 Variable을 변경한다. 
- 대표적으로 GradientDescentOptimizer(경사하강)

In [51]:
learning_rate = tf.Variable(0.01) # 학습율 설정.
# train.GradientDescentOptimizer(학습율): 학습율에 따른 오차함수 계산한다.
# minimize(오차함수): 지정된 오차 함수의 최소값을 찾는다.
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

학습시킨다.

epoch : 전체 데이터 셋에 대해 한 번 학습을 완료한 상태

- 한 번의 epoch는 인공 신경망에서 전체 데이터 셋에 대해 forward pass/backward pass 과정을 거친 것.
- 너무 작다면 underfitting이 너무 크다면 overfitting

In [55]:
# Session 만들고 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer()) 

for i in range(5001):
    # placeholder에 데이터를 대입하고 학습시킨다.
    # 학습할 데이터는 placeholder에 feed_dict라는 딕셔너리로 대입한다.
    sess.run(train, feed_dict={x: xData, y: yData}) # feed_dict의 key: placeholder 이름, value : placeholder에 저장할 리스트.
    
    # 일정 epoch 마다 중간 결과를 출력한다.
    if i % 500 == 0:
        # epoch(i), 오차(cost), 기울기, y절편
#         print('epoch : {:4d}, 오차: {:14.2f} 기울기: {:9.3f} y절편: {:9.3f}'.format(i, 
#                 sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a)[0], sess.run(b)[0]))
        print('epoch => %4d, 손실: %14.2f 기울기: %9.3f y절편: %9.3f' % (i, 
                        sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a), sess.run(b)))

print('기울기: {:9.3f} y절편: {:9.3f}'.format(sess.run(a)[0], sess.run(b)[0]))

print('8시간 근무했을 때 기대매출: {:,.0f}원'.format(sess.run(a)[0]*8+ sess.run(b)[0]))


epoch =>    0, 손실:  4583906816.00 기울기: 10386.393 y절편:  2086.925
epoch =>  500, 손실:     8687495.00 기울기: 25537.133 y절편:  1952.111
epoch => 1000, 손실:     8633814.00 기울기: 25627.451 y절편:  1504.865
epoch => 1500, 손실:     8632677.00 기울기: 25640.611 y절편:  1439.692
epoch => 2000, 손실:     8632651.00 기울기: 25642.531 y절편:  1430.192
epoch => 2500, 손실:     8632654.00 기울기: 25642.805 y절편:  1428.821
epoch => 3000, 손실:     8632657.00 기울기: 25642.844 y절편:  1428.633
epoch => 3500, 손실:     8632656.00 기울기: 25642.848 y절편:  1428.613
epoch => 4000, 손실:     8632656.00 기울기: 25642.848 y절편:  1428.613
epoch => 4500, 손실:     8632656.00 기울기: 25642.848 y절편:  1428.613
epoch => 5000, 손실:     8632656.00 기울기: 25642.848 y절편:  1428.613
기울기: 25642.848 y절편:  1428.613
8시간 근무했을 때 기대매출: 206,571원


학습이 완료된 후 원하는 입력에 대한 예측 결과를 출력한다.


In [58]:
for i in range(8, 25):
    print('{:2d}시간 근무를 했을 때 매출 기대금액: {:,.0f}원'.format(i, sess.run(Y, feed_dict={x:[i]})[0]))

 8시간 근무를 했을 때 매출 기대금액: 206,571원
 9시간 근무를 했을 때 매출 기대금액: 232,214원
10시간 근무를 했을 때 매출 기대금액: 257,857원
11시간 근무를 했을 때 매출 기대금액: 283,500원
12시간 근무를 했을 때 매출 기대금액: 309,143원
13시간 근무를 했을 때 매출 기대금액: 334,786원
14시간 근무를 했을 때 매출 기대금액: 360,428원
15시간 근무를 했을 때 매출 기대금액: 386,071원
16시간 근무를 했을 때 매출 기대금액: 411,714원
17시간 근무를 했을 때 매출 기대금액: 437,357원
18시간 근무를 했을 때 매출 기대금액: 463,000원
19시간 근무를 했을 때 매출 기대금액: 488,643원
20시간 근무를 했을 때 매출 기대금액: 514,286원
21시간 근무를 했을 때 매출 기대금액: 539,928원
22시간 근무를 했을 때 매출 기대금액: 565,571원
23시간 근무를 했을 때 매출 기대금액: 591,214원
24시간 근무를 했을 때 매출 기대금액: 616,857원
